## Get data

In [1]:
import pandas as pd
df = pd.read_csv ('data/historical_data.csv')
print(df)

              Date          High           Low          Open         Close  \
0       2022-05-01  38627.859375  37585.789062  37713.265625  38469.093750   
1       2022-05-02  39074.972656  38156.562500  38472.187500  38529.328125   
2       2022-05-03  38629.996094  37585.621094  38528.109375  37750.453125   
3       2022-05-04  39902.949219  37732.058594  37748.011719  39698.371094   
4       2022-05-05  39789.281250  35856.515625  39695.746094  36575.140625   
...            ...           ...           ...           ...           ...   
128323  2022-05-09      0.011089      0.006107      0.008104      0.006159   
128324  2022-05-10      0.007430      0.006019      0.006157      0.006023   
128325  2022-05-11      0.006550      0.004862      0.006023      0.004865   
128326  2022-05-12      0.004941      0.004618      0.004865      0.004663   
128327  2022-05-13      0.009424      0.004643      0.004666      0.006798   

              Volume     Adj Close ticker  
0       2.700276e+1

In [2]:
df.Date.unique()

array(['2022-05-01', '2022-05-02', '2022-05-03', '2022-05-04',
       '2022-05-05', '2022-05-06', '2022-05-07', '2022-05-08',
       '2022-05-09', '2022-05-10', '2022-05-11', '2022-05-13',
       '2022-05-12'], dtype=object)

In [3]:
df_date = df[df['Date']=='2022-05-12']
print(df_date.shape)
df_date.head()

(9726, 8)


,Date,High,Low,Open,Close,Volume,Adj Close,ticker
1193,2022-05-12,1.275409,1.053248,1.226878,1.134983,69722642.0,1.134983,GLMR
1242,2022-05-12,2.200926,1.712324,2.138495,1.842947,20852633.0,1.842947,SCRT
1338,2022-05-12,0.286789,0.206049,0.264309,0.275917,3879599.0,0.275917,CHSB
1351,2022-05-12,1.193851,0.979395,1.150548,1.036172,75239860.0,1.036172,SRM
1411,2022-05-12,0.028293,0.021622,0.026671,0.025651,37706601.0,0.025651,IOTX


In [4]:
df_random = df_date.sample(n = 100)
print(df_random.shape)
df_random.head()

(100, 8)


,Date,High,Low,Open,Close,Volume,Adj Close,ticker
9029,2022-05-12,0.012013,0.010540,0.011575,0.011619,0.0,0.011619,PEX
46394,2022-05-12,0.005339,0.003926,0.004501,0.004500,258411.0,0.004500,IME
41746,2022-05-12,5.314900,4.251300,5.089800,4.743700,729819.0,4.743700,NPICK
64391,2022-05-12,0.018072,0.016602,0.017685,0.017128,96186.0,0.017128,MNFT
10316,2022-05-12,0.184432,0.160928,0.180512,0.172518,2173410.0,0.172518,OPUL


In [5]:
price='Open'
data = df_random[["ticker", price]]
data.columns = ['Símbolo', 'Precio']
data.head()

,Símbolo,Precio
9029,PEX,0.011575
46394,IME,0.004501
41746,NPICK,5.089800
64391,MNFT,0.017685
10316,OPUL,0.180512


In [6]:
from datetime import date
today = date.today().strftime("%Y-%m-%d")
today

'2022-05-15'

In [7]:
from datetime import date
import pandas as pd

#NOTA: también se podría modificar la función para pasar una lista de monedas
def get_data(data_dir, date=date.today().strftime("%Y-%m-%d"), tam_data=100, price='Open'):
    """
    Obtiene datos de un directorio de una fecha dada
    param:
    data_dir: directorio de datos
    date: fecha a analizar, por default toma la fecha actual si no se especifica
    tam_data: analiza cierto tamaño del dataset de forma aleatoria, por default son 100 monedas a analizar
    price: selecciona con que precio se hará el ejercicio(High,Low,Open,Close), por default es Open
    return:
    dataframe: con datos especificados
    """
    df = pd.read_csv(data_dir)
    df_date = df[df['Date']==date]
    df_date = df_date[df_date[price] > 0]
    
    if(tam_data == None):
        df_random = df_date
    else:
        df_random = df_date.sample(n = tam_data)
    
    
    df_random = df_random.reset_index()
    data = df_random[["ticker", price]]
    data.columns = ['Símbolo', 'Precio']
    
    return data 

In [8]:
#Visualización de datos
df = get_data('data/historical_data.csv', '2022-05-12')
df.head(15)

,Símbolo,Precio
0,CWEB,0.012065
1,CXT,0.000289
2,WAULT,5.267537
3,STRP,0.850461
4,GFloki,0.002588
5,mABNB,128.523987
6,UNIFI,0.000113
7,DRF,0.001622
8,SAGA,0.057873
9,ECH,0.000760


## Transform data

### Exchange Rate Matrix Representation

In [9]:
import random
import numpy as np 

n = df.shape[0]

max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

c1 = df[['Precio']]
aux = c1.copy()
random.seed(10)
for i in range(n):
    c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
c1.drop(columns=['Precio'],inplace=True)
for i in range(len(c1.index)):
    for j in range(len(c1.columns)):
        if i==j:
            c1.loc[i,j] = 1
c1

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,42.642656,0.002357,0.014333,4.851482,9.773919e-05,110.258469,7.497940,0.213901,16.135169,...,5.374009e+02,0.192931,123.069818,0.699763,0.062188,42.871804,0.000497,7.101586,0.510589,112.940668
1,0.024638,1.000000,0.000056,0.000343,0.116210,2.341204e-06,2.641085,0.179603,0.005124,0.386495,...,1.287268e+01,0.004621,2.947963,0.016762,0.001490,1.026933,0.000012,0.170108,0.012230,2.705334
2,449.070152,18617.635911,1.000000,6.257569,2118.140375,4.267259e-02,48138.465205,3273.574500,93.388444,7044.558839,...,2.346274e+05,84.233173,53731.855793,305.514076,27.150877,18717.680838,0.217185,3100.527756,222.921528,49309.504103
3,72.503837,3005.877893,0.166120,1.000000,341.980655,6.889628e-03,7772.111832,528.529254,15.077868,1137.366945,...,3.788136e+04,13.599720,8675.182942,49.326242,4.383597,3022.030472,0.035065,500.590294,35.991406,7961.179872
4,0.220633,9.147053,0.000506,0.003074,1.000000,2.096552e-05,23.650966,1.608344,0.045883,3.461071,...,1.152751e+02,0.041385,26.399062,0.150102,0.013340,9.196206,0.000107,1.523324,0.109524,24.226311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.024638,1.021444,0.000056,0.000343,0.116210,2.341204e-06,2.641085,0.179603,0.005124,0.386495,...,1.287268e+01,0.004621,2.947963,0.016762,0.001490,1.000000,0.000012,0.170108,0.012230,2.705334
96,2118.333247,87822.263369,4.853502,29.517920,9991.595213,2.012932e-01,227076.573507,15441.956394,440.527711,33230.271803,...,1.106774e+06,397.340884,253461.460599,1441.157072,128.074877,88294.190739,1.000000,14625.668183,1051.555268,232600.544812
97,0.146037,6.054444,0.000335,0.002035,0.688818,1.387710e-05,15.654601,1.064565,0.030370,2.290886,...,7.630069e+01,0.027393,17.473568,0.099353,0.008829,6.086979,0.000071,1.000000,0.072494,16.035422
98,2.058931,85.359534,0.004717,0.028690,9.711409,1.956485e-04,220.708846,15.008930,0.428174,32.298422,...,1.075737e+03,0.386199,246.353843,1.400744,0.124483,85.818227,0.000996,14.215532,1.000000,226.077913


### Log-Transformed Representations

In [10]:
df_ln = np.round(-np.log(c1),2)
df_ln

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.00,-3.75,6.05,4.25,-1.58,9.23,-4.70,-2.01,1.54,-2.78,...,-6.29,1.65,-4.81,0.36,2.78,-3.76,7.61,-1.96,0.67,-4.73
1,3.70,-0.00,9.78,7.98,2.15,12.96,-0.97,1.72,5.27,0.95,...,-2.56,5.38,-1.08,4.09,6.51,-0.03,11.34,1.77,4.40,-1.00
2,-6.11,-9.83,-0.00,-1.83,-7.66,3.15,-10.78,-8.09,-4.54,-8.86,...,-12.37,-4.43,-10.89,-5.72,-3.30,-9.84,1.53,-8.04,-5.41,-10.81
3,-4.28,-8.01,1.80,-0.00,-5.83,4.98,-8.96,-6.27,-2.71,-7.04,...,-10.54,-2.61,-9.07,-3.90,-1.48,-8.01,3.35,-6.22,-3.58,-8.98
4,1.51,-2.21,7.59,5.78,-0.00,10.77,-3.16,-0.48,3.08,-1.24,...,-4.75,3.18,-3.27,1.90,4.32,-2.22,9.15,-0.42,2.21,-3.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.70,-0.02,9.78,7.98,2.15,12.96,-0.97,1.72,5.27,0.95,...,-2.56,5.38,-1.08,4.09,6.51,-0.00,11.34,1.77,4.40,-1.00
96,-7.66,-11.38,-1.58,-3.38,-9.21,1.60,-12.33,-9.64,-6.09,-10.41,...,-13.92,-5.98,-12.44,-7.27,-4.85,-11.39,-0.00,-9.59,-6.96,-12.36
97,1.92,-1.80,8.00,6.20,0.37,11.19,-2.75,-0.06,3.49,-0.83,...,-4.33,3.60,-2.86,2.31,4.73,-1.81,9.56,-0.00,2.62,-2.77
98,-0.72,-4.45,5.36,3.55,-2.27,8.54,-5.40,-2.71,0.85,-3.48,...,-6.98,0.95,-5.51,-0.34,2.08,-4.45,6.91,-2.65,-0.00,-5.42


In [11]:
a = []

In [12]:
# Covert to formatto use un graph
for i in range(n):
    for j in range(n):
        if (i != j):# or abs(df_ln.loc[i][j]) != 0.0):
            a.append([str(i), str(j), df_ln.loc[i][j]])
            
a

[['0', '1', -3.75],
 ['0', '2', 6.05],
 ['0', '3', 4.25],
 ['0', '4', -1.58],
 ['0', '5', 9.23],
 ['0', '6', -4.7],
 ['0', '7', -2.01],
 ['0', '8', 1.54],
 ['0', '9', -2.78],
 ['0', '10', 2.06],
 ['0', '11', -0.21],
 ['0', '12', 4.67],
 ['0', '13', -4.27],
 ['0', '14', -4.6],
 ['0', '15', -8.04],
 ['0', '16', 2.56],
 ['0', '17', 1.4],
 ['0', '18', 2.75],
 ['0', '19', 3.6],
 ['0', '20', 0.01],
 ['0', '21', -0.07],
 ['0', '22', 8.35],
 ['0', '23', 0.84],
 ['0', '24', 7.8],
 ['0', '25', -2.2],
 ['0', '26', 2.77],
 ['0', '27', 0.29],
 ['0', '28', -1.96],
 ['0', '29', 7.26],
 ['0', '30', -6.4],
 ['0', '31', -4.56],
 ['0', '32', 0.63],
 ['0', '33', -0.62],
 ['0', '34', -4.3],
 ['0', '35', -4.84],
 ['0', '36', 1.58],
 ['0', '37', 0.19],
 ['0', '38', 3.42],
 ['0', '39', -3.34],
 ['0', '40', 1.48],
 ['0', '41', 0.21],
 ['0', '42', 0.64],
 ['0', '43', -7.79],
 ['0', '44', -1.48],
 ['0', '45', 1.41],
 ['0', '46', 3.95],
 ['0', '47', -5.79],
 ['0', '48', -0.81],
 ['0', '49', 1.41],
 ['0', '50', 7.

In [13]:
#from bellman_ford import bf_negative_cycle
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt
import time

In [14]:
G = nx.DiGraph()        
G.add_weighted_edges_from(a)

In [15]:
import random
import numpy as np 

def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
    c1.drop(columns=['Precio'],inplace=True)
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1
    
    return c1

def log_transformed_rep(data):
    """
    Log-Transformed Representations
    param:
        dataframe
    return:
        dataframe
    """
    
    df_ln = np.round(-np.log(data),2)
    return df_ln

def create_grap(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

In [16]:
exchange_rate_matrix(df)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,42.642656,0.002357,0.014333,4.851482,9.773919e-05,110.258469,7.497940,0.213901,16.135169,...,5.374009e+02,0.192931,123.069818,0.699763,0.062188,42.871804,0.000497,7.101586,0.510589,112.940668
1,0.024638,1.000000,0.000056,0.000343,0.116210,2.341204e-06,2.641085,0.179603,0.005124,0.386495,...,1.287268e+01,0.004621,2.947963,0.016762,0.001490,1.026933,0.000012,0.170108,0.012230,2.705334
2,449.070152,18617.635911,1.000000,6.257569,2118.140375,4.267259e-02,48138.465205,3273.574500,93.388444,7044.558839,...,2.346274e+05,84.233173,53731.855793,305.514076,27.150877,18717.680838,0.217185,3100.527756,222.921528,49309.504103
3,72.503837,3005.877893,0.166120,1.000000,341.980655,6.889628e-03,7772.111832,528.529254,15.077868,1137.366945,...,3.788136e+04,13.599720,8675.182942,49.326242,4.383597,3022.030472,0.035065,500.590294,35.991406,7961.179872
4,0.220633,9.147053,0.000506,0.003074,1.000000,2.096552e-05,23.650966,1.608344,0.045883,3.461071,...,1.152751e+02,0.041385,26.399062,0.150102,0.013340,9.196206,0.000107,1.523324,0.109524,24.226311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.024638,1.021444,0.000056,0.000343,0.116210,2.341204e-06,2.641085,0.179603,0.005124,0.386495,...,1.287268e+01,0.004621,2.947963,0.016762,0.001490,1.000000,0.000012,0.170108,0.012230,2.705334
96,2118.333247,87822.263369,4.853502,29.517920,9991.595213,2.012932e-01,227076.573507,15441.956394,440.527711,33230.271803,...,1.106774e+06,397.340884,253461.460599,1441.157072,128.074877,88294.190739,1.000000,14625.668183,1051.555268,232600.544812
97,0.146037,6.054444,0.000335,0.002035,0.688818,1.387710e-05,15.654601,1.064565,0.030370,2.290886,...,7.630069e+01,0.027393,17.473568,0.099353,0.008829,6.086979,0.000071,1.000000,0.072494,16.035422
98,2.058931,85.359534,0.004717,0.028690,9.711409,1.956485e-04,220.708846,15.008930,0.428174,32.298422,...,1.075737e+03,0.386199,246.353843,1.400744,0.124483,85.818227,0.000996,14.215532,1.000000,226.077913


In [17]:
log_transformed_rep(exchange_rate_matrix(df))

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.00,-3.75,6.05,4.25,-1.58,9.23,-4.70,-2.01,1.54,-2.78,...,-6.29,1.65,-4.81,0.36,2.78,-3.76,7.61,-1.96,0.67,-4.73
1,3.70,-0.00,9.78,7.98,2.15,12.96,-0.97,1.72,5.27,0.95,...,-2.56,5.38,-1.08,4.09,6.51,-0.03,11.34,1.77,4.40,-1.00
2,-6.11,-9.83,-0.00,-1.83,-7.66,3.15,-10.78,-8.09,-4.54,-8.86,...,-12.37,-4.43,-10.89,-5.72,-3.30,-9.84,1.53,-8.04,-5.41,-10.81
3,-4.28,-8.01,1.80,-0.00,-5.83,4.98,-8.96,-6.27,-2.71,-7.04,...,-10.54,-2.61,-9.07,-3.90,-1.48,-8.01,3.35,-6.22,-3.58,-8.98
4,1.51,-2.21,7.59,5.78,-0.00,10.77,-3.16,-0.48,3.08,-1.24,...,-4.75,3.18,-3.27,1.90,4.32,-2.22,9.15,-0.42,2.21,-3.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3.70,-0.02,9.78,7.98,2.15,12.96,-0.97,1.72,5.27,0.95,...,-2.56,5.38,-1.08,4.09,6.51,-0.00,11.34,1.77,4.40,-1.00
96,-7.66,-11.38,-1.58,-3.38,-9.21,1.60,-12.33,-9.64,-6.09,-10.41,...,-13.92,-5.98,-12.44,-7.27,-4.85,-11.39,-0.00,-9.59,-6.96,-12.36
97,1.92,-1.80,8.00,6.20,0.37,11.19,-2.75,-0.06,3.49,-0.83,...,-4.33,3.60,-2.86,2.31,4.73,-1.81,9.56,-0.00,2.62,-2.77
98,-0.72,-4.45,5.36,3.55,-2.27,8.54,-5.40,-2.71,0.85,-3.48,...,-6.98,0.95,-5.51,-0.34,2.08,-4.45,6.91,-2.65,-0.00,-5.42


In [18]:
G = create_grap(df)

# Iniciando perfilamiento

**Función trabajada hasta el momento**

In [19]:
import numpy as np 
import networkx as nx

def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    list_val=[]

    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                a = [distance[int(edge[0])] + edge[2]['weight'], distance[int(edge[1])],predecessors[int(edge[1])],int(edge[1])]
                list_val.append(a)
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    cycle.reverse() # reverse list
    #return cycle, list_val, predecessors
    return cycle

## %TIME

In [20]:
%time bf_negative_cycle(G,0)

CPU times: user 3.1 s, sys: 74.7 ms, total: 3.17 s
Wall time: 3.17 s


[99, 98, 99]

**Propuestas de modificaciones al código**

In [21]:
def bf_negative_cycle2(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph --> CAMBIO 1
    #edges = []
    #for edge in graph.edges().data():
    #    if ~np.isnan(edge[2]['weight']):
    #        edges.append(edge)
    
    n = len(graph.nodes()) + 1
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]
    

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    list_val=[]

    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                a = [distance[int(edge[0])] + edge[2]['weight'], distance[int(edge[1])],predecessors[int(edge[1])],int(edge[1])]
                list_val.append(a)
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    cycle.reverse() # reverse list
    return cycle#, list_val, predecessors

In [22]:
%time bf_negative_cycle2(G)

CPU times: user 2.87 s, sys: 91.7 ms, total: 2.97 s
Wall time: 2.96 s


[99, 98, 99]

In [23]:
def bf_negative_cycle3(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph --> CAMBIO 1
    #edges = []
    #for edge in graph.edges().data():
    #    if ~np.isnan(edge[2]['weight']):
    #        edges.append(edge)
    
    n = len(graph.nodes()) + 1
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]
    

    # Add a start node and add zero weighted edges to all other nodes--> cambio 2
    #for i in range(n-1):
    #    edges.append((n-1, i, {'weight': 0}))
    start_node_edges = [(n-1, i, {'weight': 0}) for i in range(n-1)]
    edges = edges + start_node_edges
    
    

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    list_val=[]

    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                a = [distance[int(edge[0])] + edge[2]['weight'], distance[int(edge[1])],predecessors[int(edge[1])],int(edge[1])]
                list_val.append(a)
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    cycle.reverse() # reverse list
    return cycle#, list_val, predecessors

In [24]:
%time bf_negative_cycle3(G)

CPU times: user 2.84 s, sys: 75.7 ms, total: 2.91 s
Wall time: 2.91 s


[99, 98, 99]

In [25]:
# The Bellman-Ford function used in this notebook
def bf_negative_cycle_orig(G):

    # Remove nan edges
    n = len(G.nodes()) + 1
    edges = [edge for edge in G.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a starting node and add edges with zero weight to all other nodes
    start_node_edges = [(n-1, i, {'weight': 0}) for i in range(n-1)]
    edges = edges + start_node_edges

    # Initialize node distances and predecessors
    d = np.ones(n) * np.inf
    d[n - 1] = 0  # Starting node has zero distance
    p = np.ones(n) * -1

    # Relax n times
    for i in range(n):  
        x = -1
        for e in edges:
            if d[int(e[0])] + e[2]['weight'] < d[int(e[1])]:
                d[int(e[1])] = d[int(e[0])] + e[2]['weight']
                p[int(e[1])] = int(e[0])
                x = int(e[1])
        if x == -1:  # If no relaxation possible, no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = p[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = p[int(v)]
    return list(reversed(cycle))

In [26]:
%time bf_negative_cycle(G)

CPU times: user 3.04 s, sys: 55.8 ms, total: 3.09 s
Wall time: 3.09 s


[99, 98, 99]

In [27]:
%timeit bf_negative_cycle(G)

2.34 s ± 11.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit bf_negative_cycle2(G)

2.34 s ± 8.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit bf_negative_cycle3(G)

2.35 s ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## cProfile

In [30]:
! pip install cProfile

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cProfile (from versions: none)
ERROR: No matching distribution found for cProfile
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [31]:
import cProfile

In [32]:
cprof = cProfile.Profile()
cprof.enable()
res = bf_negative_cycle(G,0)
cprof.disable()
cprof.print_stats(sort='cumtime')

         947812 function calls (947810 primitive calls) in 3.160 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.160    1.580 interactiveshell.py:3215(run_code)
        2    0.000    0.000    3.160    1.580 {built-in method builtins.exec}
        1    0.053    0.053    3.160    3.160 3341308294.py:3(<module>)
        1    3.045    3.045    3.107    3.107 4255061534.py:4(bf_negative_cycle)
   927837    0.054    0.000    0.054    0.000 {method 'append' of 'list' objects}
     9901    0.005    0.000    0.008    0.000 reportviews.py:726(<genexpr>)
     9900    0.003    0.000    0.003    0.000 reportviews.py:712(<lambda>)
        2    0.000    0.000    0.000    0.000 codeop.py:142(__call__)
        2    0.000    0.000    0.000    0.000 numeric.py:149(ones)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 <__array_func

## pstats

In [33]:
! pip install pstats

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pstats (from versions: none)
ERROR: No matching distribution found for pstats
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [34]:
cprof.dump_stats("BF_stats")
import pstats

In [35]:
p_bf_stats = pstats.Stats("BF_stats")
print(p_bf_stats.sort_stats("cumulative").print_stats(10))

Sun May 15 13:03:13 2022    BF_stats

         947812 function calls (947810 primitive calls) in 3.160 seconds

   Ordered by: cumulative time
   List reduced from 43 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.160    1.580 /usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3215(run_code)
        2    0.000    0.000    3.160    1.580 {built-in method builtins.exec}
        1    0.053    0.053    3.160    3.160 /tmp/ipykernel_64/3341308294.py:3(<module>)
        1    3.045    3.045    3.107    3.107 /tmp/ipykernel_64/4255061534.py:4(bf_negative_cycle)
   927837    0.054    0.000    0.054    0.000 {method 'append' of 'list' objects}
     9901    0.005    0.000    0.008    0.000 /home/myuser/.local/lib/python3.8/site-packages/networkx/classes/reportviews.py:726(<genexpr>)
     9900    0.003    0.000    0.003    0.000 /home/myuser/.local/lib/python3.8/site-packages/networkx/clas

In [36]:
print(p_bf_stats.sort_stats("cumulative").print_stats("lambda|listcomp|math"))

Sun May 15 13:03:13 2022    BF_stats

         947812 function calls (947810 primitive calls) in 3.160 seconds

   Ordered by: cumulative time
   List reduced from 43 to 2 due to restriction <'lambda|listcomp|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     9900    0.003    0.000    0.003    0.000 /home/myuser/.local/lib/python3.8/site-packages/networkx/classes/reportviews.py:712(<lambda>)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}




In [37]:
print(p_bf_stats.strip_dirs().sort_stats("cumulative").print_stats("lambda|listcomp|math"))

Sun May 15 13:03:13 2022    BF_stats

         947812 function calls (947810 primitive calls) in 3.160 seconds

   Ordered by: cumulative time
   List reduced from 43 to 2 due to restriction <'lambda|listcomp|math'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     9900    0.003    0.000    0.003    0.000 reportviews.py:712(<lambda>)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}




In [38]:
print(p_bf_stats.prim_calls)

947810


In [39]:
p_bf_stats.strip_dirs().sort_stats("cumulative").print_callers()

   Ordered by: cumulative time

Function                                                                 was called by...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3215(run_code)                                       <- 
{built-in method builtins.exec}                                          <-       2    0.000    3.160  interactiveshell.py:3215(run_code)
3341308294.py:3(<module>)                                                <-       1    0.053    3.160  {built-in method builtins.exec}
4255061534.py:4(bf_negative_cycle)                                       <-       1    3.045    3.107  3341308294.py:3(<module>)
{method 'append' of 'list' objects}                                      <-  927837    0.054    0.054  4255061534.py:4(bf_negative_cycle)
reportviews.py:726(<genexpr>)                                            <-    9901    0.005    0.008  4255061534.py:4(bf_negative_cycle)
reportviews.py:712(<l

In [40]:
p_bf_stats.strip_dirs().sort_stats("cumulative").print_callees()

   Ordered by: cumulative time

Function                                                                 called...
                                                                             ncalls  tottime  cumtime
interactiveshell.py:3215(run_code)                                       ->       2    0.000    0.000  interactiveshell.py:1142(user_global_ns)
                                                                                  2    0.000    3.160  {built-in method builtins.exec}
{built-in method builtins.exec}                                          ->       1    0.053    3.160  3341308294.py:3(<module>)
                                                                                  1    0.000    0.000  3341308294.py:4(<module>)
3341308294.py:3(<module>)                                                ->       1    3.045    3.107  4255061534.py:4(bf_negative_cycle)
4255061534.py:4(bf_negative_cycle)                                       ->       1    0.000    0.000  digra

In [41]:
p_bf_stats.strip_dirs().sort_stats("cumulative").print_callers(10)

   Ordered by: cumulative time
   List reduced from 43 to 10 due to restriction <10>

Function                             was called by...
                                         ncalls  tottime  cumtime
interactiveshell.py:3215(run_code)   <- 
{built-in method builtins.exec}      <-       2    0.000    3.160  interactiveshell.py:3215(run_code)
3341308294.py:3(<module>)            <-       1    0.053    3.160  {built-in method builtins.exec}
4255061534.py:4(bf_negative_cycle)   <-       1    3.045    3.107  3341308294.py:3(<module>)
{method 'append' of 'list' objects}  <-  927837    0.054    0.054  4255061534.py:4(bf_negative_cycle)
reportviews.py:726(<genexpr>)        <-    9901    0.005    0.008  4255061534.py:4(bf_negative_cycle)
reportviews.py:712(<lambda>)         <-    9900    0.003    0.003  reportviews.py:726(<genexpr>)
codeop.py:142(__call__)              <- 
numeric.py:149(ones)                 <-       2    0.000    0.000  4255061534.py:4(bf_negative_cycle)
{built-in metho

In [42]:
p_bf_stats.strip_dirs().sort_stats("cumulative").print_callees("BF|lambda")

   Ordered by: cumulative time
   List reduced from 43 to 1 due to restriction <'BF|lambda'>

Function                      called...
                                  ncalls  tottime  cumtime
reportviews.py:712(<lambda>)  -> 




## line_profiler

In [43]:
! pip install line_profiler

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [44]:
import line_profiler

In [45]:
line_prof = line_profiler.LineProfiler()
print(line_prof(bf_negative_cycle)(G))

[99, 98, 99]


In [46]:
print(line_prof.print_stats())

Timer unit: 1e-06 s

Total time: 6.85602 s
File: /tmp/ipykernel_64/4255061534.py
Function: bf_negative_cycle at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
     5                                               
     6         1          3.0      3.0      0.0      assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
     7                                               
     8         1          1.0      1.0      0.0      if node_ini is None:
     9         1         41.0     41.0      0.0          n_nodes = len(graph.nodes())
    10                                               else:
    11                                                   assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
    12              

In [47]:
line_prof = line_profiler.LineProfiler()
print(line_prof(bf_negative_cycle2)(G))
print(line_prof.print_stats())

[99, 98, 99]
Timer unit: 1e-06 s

Total time: 7.24417 s
File: /tmp/ipykernel_64/3046895781.py
Function: bf_negative_cycle2 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bf_negative_cycle2(graph, node_ini=None, distance_ini=np.inf):
     2                                               
     3         1          3.0      3.0      0.0      assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
     4                                               
     5         1          1.0      1.0      0.0      if node_ini is None:
     6         1         17.0     17.0      0.0          n_nodes = len(graph.nodes())
     7                                               else:
     8                                                   assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
     

In [48]:
line_prof = line_profiler.LineProfiler()
print(line_prof(bf_negative_cycle3)(G))
print(line_prof.print_stats())

[99, 98, 99]
Timer unit: 1e-06 s

Total time: 8.11544 s
File: /tmp/ipykernel_64/3000221302.py
Function: bf_negative_cycle3 at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def bf_negative_cycle3(graph, node_ini=None, distance_ini=np.inf):
     2                                               
     3         1          3.0      3.0      0.0      assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
     4                                               
     5         1          2.0      2.0      0.0      if node_ini is None:
     6         1         17.0     17.0      0.0          n_nodes = len(graph.nodes())
     7                                               else:
     8                                                   assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
     

## memory_profiler

In [49]:
from memory_profiler import memory_usage

In [50]:
t = (bf_negative_cycle, (G,1,100000))
print(memory_usage(t, max_usage=True))

312.76953125


In [51]:
start_mem = memory_usage(max_usage=True)
res = memory_usage(t, max_usage=True, retval=True)
print('start mem', start_mem)
print('max mem', res[0])
print('used mem', res[0]-start_mem)
print('fun output', res[1])

start mem 150.27734375
max mem 312.51171875
used mem 162.234375
fun output [99, 98, 99]


In [52]:
%load_ext memory_profiler

In [53]:
%memit #how much RAM this process is consuming

peak memory: 150.29 MiB, increment: 0.00 MiB


In [54]:
%memit -c bf_negative_cycle(G)

peak memory: 432.84 MiB, increment: 282.55 MiB


In [60]:
%%file BF_memory_profiler.py

import math

from pytest import approx
from scipy.integrate import quad
from memory_profiler import profile
import numpy as np 
import networkx as nx


@profile #decorate the functions you want to profile with memory_profiler 
def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    list_val=[]

    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                a = [distance[int(edge[0])] + edge[2]['weight'], distance[int(edge[1])],predecessors[int(edge[1])],int(edge[1])]
                list_val.append(a)
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    cycle.reverse() # reverse list
    return cycle#, list_val, predecessors

if __name__ == "__main__":
    edges3 = [["0","1", 5],
         ["1","2", 20],
         ["1","5", 30],
         ["1","6", 60],
         ["2","3", 10],
         ["2","4", 75],
         ["3","2", -15],
         ["4","9", 100],
         ["5","4", 25],
         ["5","6", 5],
         ["5","8", 50],
         ["6","7", -50],
         ["7","8", -10]]
        
    G3 = nx.DiGraph()        
    G3.add_weighted_edges_from(edges3)
    res,x,y = bf_negative_cycle(G3)
    print("aproximación: {:0.6e}".format(res))
    print(res == approx(obj))
    

Overwriting BF_memory_profiler.py


In [61]:
import math
edges3 = [["0","1", 5],
         ["1","2", 20],
         ["1","5", 30],
         ["1","6", 60],
         ["2","3", 10],
         ["2","4", 75],
         ["3","2", -15],
         ["4","9", 100],
         ["5","4", 25],
         ["5","6", 5],
         ["5","8", 50],
         ["6","7", -50],
         ["7","8", -10]]
        
G3 = nx.DiGraph()        
G3.add_weighted_edges_from(edges3)

In [62]:
from BF_memory_profiler import bf_negative_cycle

In [58]:
%mprun -f bf_negative_cycle bf_negative_cycle(G)

*** KeyboardInterrupt exception caught in code being profiled.


Filename: /home/myuser/.local/lib/python3.8/site-packages/memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
  1183    163.3 MiB    163.3 MiB           1               @wraps(wrapped=func)
  1184                                                     def wrapper(*args, **kwargs):
  1185    163.3 MiB      0.0 MiB           1                   prof = get_prof()
  1186    243.6 MiB     80.2 MiB           1                   val = prof(func)(*args, **kwargs)
  1187                                                         show_results_bound(prof)
  1188                                                         return val

In [59]:
%%bash
python3 BF_memory_profiler.py

Filename: BF_memory_profiler.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    11     86.7 MiB     86.7 MiB           1   @profile #decorate the functions you want to profile with memory_profiler 
    12                                         def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    13                                             
    14     86.7 MiB      0.0 MiB           1       assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    15                                             
    16     86.7 MiB      0.0 MiB           1       if node_ini is None:
    17     86.7 MiB      0.0 MiB           1           n_nodes = len(graph.nodes())
    18                                             else:
    19                                                 assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
  

Traceback (most recent call last):
  File "BF_memory_profiler.py", line 85, in <module>
    print(res == approx(obj))
NameError: name 'obj' is not defined


CalledProcessError: Command 'b'python3 BF_memory_profiler.py\n'' returned non-zero exit status 1.

Heapy

In [ ]:
#cpython


In [63]:
import math
import time

from pytest import approx
from scipy.integrate import quad
from IPython.display import HTML, display

In [64]:
%%file bf_negative.pyx
def bf_negative_cycle_cpython(G):
    
    import numpy as np 
    
    cdef unsigned int i,n

    # Remove nan edges
    n = len(G.nodes()) + 1
    edges = [edge for edge in G.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a starting node and add edges with zero weight to all other nodes
    start_node_edges = [(n-1, i, {'weight': 0}) for i in range(n-1)]
    edges = edges + start_node_edges

    # Initialize node distances and predecessors
    d = np.ones(n) * np.inf
    d[n - 1] = 0  # Starting node has zero distance
    p = np.ones(n) * -1

    # Relax n times
    for i in range(n):  
        x = -1
        for e in edges:
            if d[int(e[0])] + e[2]['weight'] < d[int(e[1])]:
                d[int(e[1])] = d[int(e[0])] + e[2]['weight']
                p[int(e[1])] = int(e[0])
                x = int(e[1])
        if x == -1:  # If no relaxation possible, no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = p[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = p[int(v)]
    return list(reversed(cycle))


Overwriting bf_negative.pyx


In [65]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize
#from bellman_ford import bf_negative_cycle
import numpy as np 
import networkx as nx
import matplotlib.pyplot as plt
import time

setup(ext_modules = cythonize("bf_negative.pyx", 
                              compiler_directives={'language_level' : 3})
     )
      

Overwriting setup.py


In [66]:
%%bash
python3 setup.py build_ext --inplace

Compiling bf_negative.pyx because it changed.
[1/1] Cythonizing bf_negative.pyx
running build_ext
building 'bf_negative' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c bf_negative.c -o build/temp.linux-x86_64-3.8/bf_negative.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/bf_negative.o -o /datos/bf_negative.cpython-38-x86_64-linux-gnu.so


In [67]:
import bf_negative

In [68]:
%time bf_negative_cycle_orig(G)

CPU times: user 1.38 s, sys: 11.4 ms, total: 1.39 s
Wall time: 1.39 s


[99, 98, 99]

In [69]:
import numpy as np 
import networkx as nx

In [70]:
%load_ext Cython

In [71]:
%%cython
def bf_negative_cycle_orig(G):
    
    import numpy as np 
    
    cdef unsigned int i,n
    
    # Remove nan edges
    n = len(G.nodes()) + 1
    edges = [edge for edge in G.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a starting node and add edges with zero weight to all other nodes
    start_node_edges = [(n-1, i, {'weight': 0}) for i in range(n-1)]
    edges = edges + start_node_edges

    # Initialize node distances and predecessors
    d = np.ones(n) * np.inf
    d[n - 1] = 0  # Starting node has zero distance
    p = np.ones(n) * -1

    # Relax n times
    for i in range(n):  
        x = -1
        for e in edges:
            if d[int(e[0])] + e[2]['weight'] < d[int(e[1])]:
                d[int(e[1])] = d[int(e[0])] + e[2]['weight']
                p[int(e[1])] = int(e[0])
                x = int(e[1])
        if x == -1:  # If no relaxation possible, no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = p[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = p[int(v)]
    return list(reversed(cycle))


In [72]:
%time bf_negative_cycle_orig(G)

CPU times: user 924 ms, sys: 13.1 ms, total: 937 ms
Wall time: 933 ms


[99, 98, 99]

In [73]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_negative.pyx

In [74]:
display(HTML("bf_negative.html"))

In [90]:
%%file bf_cython2.pyx
import numpy as np 
def bf_negative_cycle_cc(graph,  node_ini=None, unsigned int distance_ini=np.inf):
    
    cdef unsigned int i,n,n_nodes
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    cycle.reverse()
    return cycle

Overwriting bf_cython2.pyx


In [82]:
%time bf_negative_cycle_cc(G)

CPU times: user 524 ms, sys: 4.17 ms, total: 528 ms
Wall time: 527 ms


[99, 98, 99]

In [91]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_cython2.pyx

In [92]:
display(HTML("bf_cython2.html"))